In [ ]:
import polars as pl

In [ ]:
train_claims_decomposed = (
    r"/Users/maximilianheil/OMSCS/07_CLEF/Task3/checkthat-2025-numerical-data/decomposition/train_claims_decomposed.json"
)
ir_evidence_corpus = r"/Users/maximilianheil/OMSCS/07_CLEF/Task3/checkthat-2025-numerical-data/reranking/ir_evidence_corpus.csv"
reranked_results_train = (
    r"/Users/maximilianheil/OMSCS/07_CLEF/Task3/checkthat-2025-numerical-data/reranking/reranked_results_train.csv"
)

In [ ]:
ir_evidence_corpus = pl.read_csv(ir_evidence_corpus)
reranked_results_train = pl.read_csv(reranked_results_train)
train_claims_decomposed = pl.read_json(train_claims_decomposed)

In [ ]:
# Prepare reranked_results_train
reranked_results_train = reranked_results_train.with_columns(
    [
        pl.col("reranked_docno").str.strip_chars("[]").str.replace_all("'", "").str.split(", ").cast(pl.List(pl.Int64)),
        pl.col("docno").str.strip_chars("[]").str.replace_all("'", "").str.split(", ").cast(pl.List(pl.Int64)),
        pl.col("reranked_scores").str.strip_chars("[]").str.split(", ").cast(pl.List(pl.Float32)),
    ]
)

# Merge reranked results with train_claims_decomposed to get the labels (and metadata)
merged = reranked_results_train.join(train_claims_decomposed.select(["claim_id", "label"]), on="claim_id", how="left")

# explode the lists
merged = merged.explode(["docno", "reranked_docno", "reranked_scores"])

# Rename evidence_corpus columns
ir_evidence_corpus = ir_evidence_corpus.rename({"text": "evidence_text"})

# Merge with ir_corpus to get the text
merged = (
    merged.join(ir_evidence_corpus, on="docno", how="left", validate="m:m")
    .drop(pl.col("_right"))
    .rename({"evidence_text": "evidence_text_retriever"})
)
merged = (
    merged.join(ir_evidence_corpus, left_on="reranked_docno", right_on="docno", how="left", validate="m:m")
    .drop(pl.col("_right"))
    .rename({"evidence_text": "evidence_text_reranker"})
)

In [ ]:
merged